## Step 1: Import Pandas, Numpy, and Data

In [115]:
import pandas as pd
import numpy as np

In [116]:
cafeData = pd.read_csv('/Users/Marcy_Student/M1-Project--Cafe-Sales/Data/dirty_cafe_sales.csv')

In [117]:
cafeData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [118]:
# Step 1: Calculate Differences
tIDdiff = cafeData["Transaction ID"].count()
idiff = cafeData["Item"].count()
qdiff= cafeData["Quantity"].count()
ppudiff = cafeData["Price Per Unit"].count()
tsdiff = cafeData["Total Spent"].count()
PMdiff = cafeData["Payment Method"].count()
ldiff = cafeData["Location"].count()
TDdiff = cafeData["Transaction Date"].count()
diff1= 10000 - tIDdiff
diff2 = 10000 - idiff
diff3 = 10000 - qdiff
diff4 = 10000 - ppudiff
diff5 = 10000 - tsdiff
diff6 = 10000 - PMdiff
diff7 = 10000 - ldiff
diff8 = 10000 - TDdiff
print(f'Missing values for each column \n Transaction ID: {diff1}  \n Item: {diff2} \n Quantity: {diff3} \n Price Per Unit: {diff4} \n Total Spent: {diff5} \n Payment Method: {diff6} \n Location: {diff7} \n Transaction Date: {diff8}')

Missing values for each column 
 Transaction ID: 0  
 Item: 333 
 Quantity: 138 
 Price Per Unit: 179 
 Total Spent: 173 
 Payment Method: 2579 
 Location: 3265 
 Transaction Date: 159


## Step 2: Replace all missing values with 0 so it can be converted to the proper data type

In [119]:
# Replace Quantity Missing Values
cafeData['Quantity'] = cafeData['Quantity'].replace('UNKNOWN', np.nan)
cafeData['Quantity'] = cafeData['Quantity'].replace("ERROR", np.nan)
cafeData['Quantity'] = cafeData['Quantity'].fillna(np.nan)


In [120]:
# Replace PPU Missing Values
cafeData[['Price Per Unit','Total Spent' ]] = cafeData[['Price Per Unit','Total Spent' ]].replace('UNKNOWN', np.nan)
cafeData[['Price Per Unit','Total Spent' ]] = cafeData[['Price Per Unit','Total Spent' ]].replace("ERROR", np.nan)
cafeData[['Price Per Unit','Total Spent' ]] = cafeData[['Price Per Unit','Total Spent' ]].fillna(np.nan)

In [121]:
# Replace Transaction Date Missing Values
cafeData['Transaction Date'] = cafeData['Transaction Date'].replace(['UNKNOWN', 'ERROR'], pd.NA)
cafeData['Valid Date'] = cafeData['Transaction Date'].notna()
cafeData['Transaction Date'] = pd.to_datetime(cafeData['Transaction Date'], errors='coerce')

In [122]:
# Replace Item Missing Values
cafeData['Item'] = cafeData['Item'].replace('UNKNOWN', "No Data")
cafeData['Item'] = cafeData['Item'].replace("ERROR", "No Data")
cafeData['Item'] = cafeData['Item'].replace("nan", "No Data")
cafeData['Item'] = cafeData['Item'].fillna("No Data")


## Step 3: Convert Data Types

In [123]:
cafeData[['Transaction ID','Item','Payment Method','Location']] = cafeData[['Transaction ID','Item','Payment Method','Location']].astype(str)
cafeData['Quantity'] = cafeData['Quantity'].astype(float)
cafeData[['Price Per Unit','Total Spent' ]] = cafeData[['Price Per Unit','Total Spent' ]].astype(float)
cafeData['Transaction Date'] = pd.to_datetime(cafeData['Transaction Date'])
cafeData.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    10000 non-null  object        
 1   Item              10000 non-null  object        
 2   Quantity          9521 non-null   float64       
 3   Price Per Unit    9467 non-null   float64       
 4   Total Spent       9498 non-null   float64       
 5   Payment Method    10000 non-null  object        
 6   Location          10000 non-null  object        
 7   Transaction Date  9540 non-null   datetime64[ns]
 8   Valid Date        10000 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(3), object(4)
memory usage: 634.9+ KB


## Step 3.5: Check Counts for fillable columns and make sure there are no duplicates that will interfere with EDA

In [124]:
print(cafeData['Transaction ID'].duplicated().sum())

0


In [125]:
# Count 'Total Spent' Values
# Note : 0's are missing values
cafeData['Total Spent'].value_counts()

Total Spent
6.0     979
12.0    939
3.0     930
4.0     923
20.0    746
15.0    734
8.0     677
10.0    524
2.0     497
9.0     479
5.0     468
16.0    444
25.0    259
7.5     237
1.0     232
4.5     225
1.5     205
Name: count, dtype: int64

In [126]:
# Count 'Quantity' Values
# Note : 0's are missing values
cafeData['Quantity'].value_counts()

Quantity
5.0    2013
2.0    1974
4.0    1863
3.0    1849
1.0    1822
Name: count, dtype: int64

In [127]:
# Count 'Price Per Unit' Values
# Note : 0's are missing values
cafeData['Price Per Unit'].value_counts()

Price Per Unit
3.0    2429
4.0    2331
2.0    1227
5.0    1204
1.0    1143
1.5    1133
Name: count, dtype: int64

In [128]:
# Count 'Item' Values
# Note : 0's are missing values
cafeData['Item'].value_counts()

Item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
No Data      969
Name: count, dtype: int64

## Step 4: Fill in what we can with the data we have

In [129]:
# Use PPU, Quantity, and Total Spent to calculate each other
for index, row in cafeData.iterrows():
    qty = row['Quantity']
    ppu = row['Price Per Unit']
    total = row['Total Spent']
    

    if (pd.isna(total) or total == 0) and qty > 0 and ppu > 0:
        cafeData.at[index, 'Total Spent'] = qty * ppu
    

    elif (pd.isna(ppu) or ppu == 0) and qty > 0 and total > 0:
        cafeData.at[index, 'Price Per Unit'] = total / qty


    elif (pd.isna(qty) or qty == 0) and ppu > 0 and total > 0:
        cafeData.at[index, 'Quantity'] = total / ppu


In [130]:
# Defines Item by PPU
cafeData.loc[cafeData['Price Per Unit'] == 1, 'Item'] = 'Cookie'
cafeData.loc[cafeData['Price Per Unit'] == 1.5, 'Item'] = 'Tea'
cafeData.loc[cafeData['Price Per Unit'] == 2, 'Item'] = 'Coffee'
cafeData.loc[(cafeData['Item'] == 'No Data') & (cafeData['Price Per Unit'] == 3.0), 'Item'] = 'Cake, Juice'
cafeData.loc[(cafeData['Item'] == 'No Data') & (cafeData['Price Per Unit'] == 4.0), 'Item'] = 'Sandwich, Smoothie'
cafeData.loc[cafeData['Price Per Unit'] == 5, 'Item'] = 'Salad'




In [131]:
#Defines PPU by Item

cafeData.loc[cafeData['Item'] == 'Cookie', 'Price Per Unit'] = 1
cafeData.loc[cafeData['Item'] == 'Tea', 'Price Per Unit'] = 1.5
cafeData.loc[cafeData['Item'] == 'Coffee', 'Price Per Unit'] = 2
cafeData.loc[cafeData['Item'] == 'Cake', 'Price Per Unit'] = 3
cafeData.loc[cafeData['Item'] == 'Juice', 'Price Per Unit'] = 3
cafeData.loc[cafeData['Item'] == 'Sandwich', 'Price Per Unit'] = 4
cafeData.loc[cafeData['Item'] == 'Smoothie', 'Price Per Unit'] = 4
cafeData.loc[cafeData['Item'] == 'Salad', 'Price Per Unit'] = 5

In [132]:
# Use PPU, Quantity, and Total Spent to calculate each other
for index, row in cafeData.iterrows():
    qty = row['Quantity']
    ppu = row['Price Per Unit']
    total = row['Total Spent']
    

    if (pd.isna(total) or total == 0) and qty > 0 and ppu > 0:
        cafeData.at[index, 'Total Spent'] = qty * ppu
    

    elif (pd.isna(ppu) or ppu == 0) and qty > 0 and total > 0:
        cafeData.at[index, 'Price Per Unit'] = total / qty


    elif (pd.isna(qty) or qty == 0) and ppu > 0 and total > 0:
        cafeData.at[index, 'Quantity'] = total / ppu


## Step 5: Make sure Data is cleaned as much as it can be without making assumptions

In [133]:
# Looks at where there is no Item Data to make sure it can't be calculated
print(cafeData[cafeData['Item'] == 'No Data'])

     Transaction ID     Item  Quantity  Price Per Unit  Total Spent  \
1761    TXN_3611851  No Data       4.0             NaN          NaN   
2289    TXN_7524977  No Data       4.0             NaN          NaN   
3779    TXN_7376255  No Data       NaN             NaN         25.0   
4152    TXN_9646000  No Data       2.0             NaN          NaN   
7597    TXN_1082717  No Data       NaN             NaN          9.0   
9819    TXN_1208561  No Data       NaN             NaN         20.0   

      Payment Method  Location Transaction Date  Valid Date  
1761     Credit Card       nan       2023-02-09        True  
2289           ERROR       nan       2023-12-09        True  
3779             nan  In-store       2023-05-27        True  
4152             nan  In-store       2023-12-14        True  
7597  Digital Wallet  In-store       2023-12-13        True  
9819     Credit Card       nan       2023-08-19        True  


In [134]:
print(cafeData[cafeData['Total Spent'] == 0])

Empty DataFrame
Columns: [Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Payment Method, Location, Transaction Date, Valid Date]
Index: []


In [135]:
print(cafeData[cafeData['Quantity'] == 0])

Empty DataFrame
Columns: [Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Payment Method, Location, Transaction Date, Valid Date]
Index: []


In [136]:
print(cafeData[cafeData['Price Per Unit'] == 0])

Empty DataFrame
Columns: [Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Payment Method, Location, Transaction Date, Valid Date]
Index: []


In [137]:
# Count 'Item' Values
# Note : 0's are missing values
cafeData['Item'].value_counts()

Item
Coffee                1291
Salad                 1272
Cookie                1213
Tea                   1207
Juice                 1171
Cake                  1139
Sandwich              1131
Smoothie              1096
Cake, Juice            247
Sandwich, Smoothie     227
No Data                  6
Name: count, dtype: int64

In [138]:
# Count 'Quantity' Values
# Note : 0's are missing values
cafeData['Quantity'].value_counts()

Quantity
5.0    2108
2.0    2056
3.0    1946
4.0    1941
1.0    1926
Name: count, dtype: int64

In [139]:
# Count 'Price Per Unit' Values
# Note : 0's are missing values
cafeData['Price Per Unit'].value_counts()

Price Per Unit
3.0    2557
4.0    2454
2.0    1291
5.0    1272
1.0    1213
1.5    1207
Name: count, dtype: int64

In [140]:
# Count 'Total Spent' Values
# Note : 0's are missing values
cafeData['Total Spent'].value_counts()

Total Spent
6.0     1019
12.0     996
4.0      969
3.0      968
20.0     789
15.0     766
8.0      720
10.0     542
2.0      518
9.0      510
5.0      496
16.0     466
25.0     269
7.5      250
1.0      249
4.5      238
1.5      212
Name: count, dtype: int64

## Step 6: Export the data


In [141]:
# Export the dataframe as a new csv file
cafeData.to_csv('clean_cafe_sales.csv', index=False)